In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from mpl_toolkits.mplot3d import Axes3D

## Variáveis globais

In [27]:
category_field = "client_id"
paths_or1 = [r"C:\Users\ullma\Documents\TCC\tcc-gabriel\data\uci-online-retail-source\online_retail.csv",
             r"C:\Users\ullma\Desktop\all_tcc_results\online_retail1\best-seller\online_retail_g_amounts.csv",
             r"C:\Users\ullma\Desktop\all_tcc_results\online_retail1\categ-cf\c_cbr.csv",
             r"C:\Users\ullma\Desktop\all_tcc_results\online_retail1\categ-cf\best_selling_by_client.csv"]

paths_or2 = [r"C:\Users\ullma\Documents\TCC\tcc-gabriel\data\uci-online-retail-2\online_retail_II.csv",
             r"C:\Users\ullma\Desktop\all_tcc_results\online_retail2\best-seller\online_retail2_g_amounts.csv",
             r"C:\Users\ullma\Desktop\all_tcc_results\online_retail2\categ-cf\c_cbr.csv",
             r"C:\Users\ullma\Desktop\all_tcc_results\online_retail2\categ-cf\best_selling_by_country.csv"]

paths_olist = [r"C:\Users\ullma\Documents\TCC\tcc-gabriel\data\olist-processed\olist_processed.csv", 
               r"C:\Users\ullma\Desktop\all_tcc_results\opa.csv",
               r"C:\Users\ullma\Desktop\all_tcc_results\olist\categ-cf\c_cbr.csv",
               r"C:\Users\ullma\Desktop\all_tcc_results\olist\categ-cf\best_selling_by_country.csv"]

ds_list = {"or1": paths_or1, "or2": paths_or2, "olist": paths_olist}

ds_selected = "olist"

In [28]:
def getLabelName(name):
    try:
        names = {
            "United Kingdom": "Reino Unido",
            "Brazil": "Brasil",
            "USA": "EUA",
            "France": "França",
            "Germany": "Alemanha",
            "Japan": "Japão",
            "RS": "Rio Grande do Sul",
            "SP": "São Paulo",
            "RJ": "Rio de Janeiro",
            "MG": "Minas Gerais",
            "BA": "Bahia",
            "PR": "Paraná",
        }
        return names[name]
    except KeyError:
        return name

## Análise por categoria

In [29]:
base_ds = pd.read_csv(ds_list[ds_selected][0], sep=",")
print(len(base_ds))
base_ds = base_ds.dropna()
encoder = OrdinalEncoder()
encoded_clients = encoder.fit_transform(
    base_ds['clientId'].values.reshape(-1, 1))
encoded_prods = encoder.fit_transform(
    base_ds['product_id'].values.reshape(-1, 1))
base_ds['clientId'] = encoded_clients
base_ds['product_id'] = encoded_prods
base_ds[:5]

112650


,clientId,country,product_id
0,50398.0,RJ,8361.0
1,87747.0,SP,28636.0
2,21004.0,MG,24819.0
3,65566.0,SP,14831.0
4,37639.0,SP,21360.0


In [30]:
processed_amounts_ds = pd.read_csv(ds_list[ds_selected][1], sep=",")
processed_amounts_ds[:5]

,client_id,product_id,rating,amount
0,0,6940,1,31
1,1,10124,1,7
2,2,12581,1,5
3,3,4543,1,3
4,4,24781,1,1


In [31]:
processed_ds = pd.read_csv(ds_list[ds_selected][2], sep=",")
processed_ds[:5]

,client_id,product_id,rating
0,0,6940,1
1,1,10124,1
2,2,12581,1
3,3,4543,1
4,4,24781,1


In [32]:
categories = pd.read_csv(ds_list[ds_selected][3], sep=",")
categories = categories['country'].unique()
categories

array(['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG',
       'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR',
       'RS', 'SC', 'SE', 'SP', 'TO'], dtype=object)

In [33]:
filtered_by_categ = []
col_clis = {}
for category in categories:
    filtered_by_categ = base_ds[(base_ds.country == category)]
    col_clis[category] = [];
    #print(filtered_by_categ)
    for row in filtered_by_categ.values:
        #print(row)
        if ds_selected == "olist":
            col_clis[category].append(int(row[0]))
        else:
            col_clis[category].append(int(row[6]))

In [34]:
for category in categories:
    col_clis[category] = set(col_clis[category])
len(col_clis)

27

In [35]:
def buildFilter(col_clis, dsname, fieldname, keyname):
    filterStr = "("
    ds_field = fieldname
    ccl = len(col_clis[keyname])
    count = 1
    for cli in col_clis[keyname]:
        filterStr += ds_field + "==" + str(cli)
        if count < ccl:
            filterStr += ") | ("
        count += 1
    filterStr += ")"
    return filterStr

In [36]:
for category in categories:
    npd = processed_ds[processed_ds[category_field].isin(col_clis[category])]
    sns_plot = sns.displot(npd["rating"], kind="kde")
    sns_plot.set(title=getLabelName(category))
    sns_plot.set_axis_labels('Avaliação', 'Densidade')
    sns_plot.savefig("temp_img/ratings_" + category + ".png")
    print("Saved: " + category + " | ", end="", flush=True)
    plt.close()

Saved: AC | Saved: AL | Saved: AM | Saved: AP | Saved: BA | Saved: CE | Saved: DF | Saved: ES | Saved: GO | Saved: MA | Saved: MG | Saved: MS | Saved: MT | Saved: PA | Saved: PB | Saved: PE | Saved: PI | Saved: PR | Saved: RJ | Saved: RN | Saved: RO | Saved: RR | Saved: RS | Saved: SC | Saved: SE | Saved: SP | Saved: TO | 

In [26]:
for category in categories:
    npd = processed_amounts_ds[processed_amounts_ds[category_field].isin(col_clis[category])]

    #plt.xlim(0, 1300)
    #plt.ylim(1, 1000000)
    #plt.yscale("log")

    plt.xlabel('Frequência de pedidos')
    plt.ylabel('Contagem')
    sns_plot = sns.histplot(npd["amount"], kde=False, stat="count") #bins=12
    sns_plot.set(title=getLabelName(category))
    plt.savefig("temp_img/sales_volume_" + category + ".png")
    print("Saved: " + category + " | ", end="", flush=True)
    plt.close()

1      862
2      210
3      172
4      146
5      120
      ... 
500      1
400      1
166      1
134      1
610      1
Name: amount, Length: 138, dtype: int64


In [25]:
for category in categories:
    npd = processed_amounts_ds[processed_amounts_ds[category_field].isin(col_clis[category])]
    sns_plot = sns.lmplot(x="product_id", y="amount", data=npd, fit_reg=False)
    sns_plot.set(title=getLabelName(category))
    sns_plot.set_axis_labels('ID Produto', 'Frequência de pedidos')
    sns_plot.savefig("temp_img/product_amount_" + category + ".png")
    print("Saved: " + category + " | ", end="", flush=True)
    plt.close()

Saved: United Kingdom | Saved: Iceland | Saved: Finland | Saved: Italy | Saved: Norway | Saved: Bahrain | Saved: Spain | Saved: Portugal | Saved: Switzerland | Saved: Austria | Saved: Cyprus | Saved: Belgium | Saved: Unspecified | Saved: Denmark | Saved: Australia | Saved: France | Saved: Germany | Saved: RSA | Saved: Greece | Saved: Sweden | Saved: Israel | Saved: USA | Saved: Saudi Arabia | Saved: Poland | Saved: United Arab Emirates | Saved: Singapore | Saved: Japan | Saved: Netherlands | Saved: Lebanon | Saved: Brazil | Saved: Czech Republic | Saved: EIRE | Saved: Channel Islands | Saved: European Community | Saved: Lithuania | Saved: Canada | Saved: Malta | 

In [37]:
for category in categories:
    npd = processed_amounts_ds[processed_amounts_ds[category_field].isin(col_clis[category])]
    sns_plot = sns.lmplot(x="rating", y="amount", data=npd, fit_reg=False)
    sns_plot.set(title=getLabelName(category))
    sns_plot.set_axis_labels('Avaliação', 'Frequência de pedidos')
    sns_plot.savefig("temp_img/rating_amount_" + category + ".png")
    print("Saved: " + category + " | ", end="", flush=True)
    plt.close()

Saved: AC | Saved: AL | Saved: AM | Saved: AP | Saved: BA | Saved: CE | Saved: DF | Saved: ES | Saved: GO | Saved: MA | Saved: MG | Saved: MS | Saved: MT | Saved: PA | Saved: PB | Saved: PE | Saved: PI | Saved: PR | Saved: RJ | Saved: RN | Saved: RO | Saved: RR | Saved: RS | Saved: SC | Saved: SE | Saved: SP | Saved: TO | 

In [61]:
for category in categories:
    npd = processed_amounts_ds[processed_amounts_ds[category_field].isin(col_clis[category])]
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(npd['product_id'], npd['amount'], npd['rating'], c='navy', s=60)
    ax.view_init(15, 230)

    ax.set_title(getLabelName(category))
    ax.set_xlabel('ID Produto', labelpad=10)
    ax.set_ylabel('Frequência', labelpad=10)
    ax.set_zlabel('Avaliação')

    plt.tight_layout()
    for label in ax.get_yaxis().get_ticklabels()[::2]:
        label.set_visible(False)
        
    for label in ax.get_xaxis().get_ticklabels()[::2]:
        label.set_visible(False)
        
    plt.savefig("temp_img/disp3d2_" + category + ".png")
    print("Saved: " + category + " | ", end="", flush=True)
    plt.close()

Saved: Australia | Saved: Austria | Saved: Bahrain | Saved: Belgium | Saved: Bermuda | Saved: Brazil | Saved: Canada | Saved: Channel Islands | Saved: Cyprus | Saved: Czech Republic | Saved: Denmark | Saved: EIRE | Saved: European Community | Saved: Finland | Saved: France | Saved: Germany | Saved: Greece | Saved: Hong Kong | Saved: Iceland | Saved: Israel | Saved: Italy | Saved: Japan | Saved: Korea | Saved: Lebanon | Saved: Lithuania | Saved: Malta | Saved: Netherlands | Saved: Nigeria | Saved: Norway | Saved: Poland | Saved: Portugal | Saved: RSA | Saved: Saudi Arabia | Saved: Singapore | Saved: Spain | Saved: Sweden | Saved: Switzerland | Saved: Thailand | Saved: USA | Saved: United Arab Emirates | Saved: United Kingdom | Saved: Unspecified | Saved: West Indies | 

## Analisando produtos mais comuns em um intervalo
Conforme análise 2, categorias

In [157]:
un_prod = base_ds[(base_ds.product_id >= 1000) & (base_ds.product_id <= 2000)]['desc'].unique()
len(un_prod)

1088

In [161]:
un_prod[:5]

array(['SET 7 BABUSHKA NESTING BOXES', 'HAND WARMER UNION JACK',
       'HAND WARMER RED POLKA DOT', "POPPY'S PLAYHOUSE BEDROOM ",
       "POPPY'S PLAYHOUSE KITCHEN"], dtype=object)